In [2]:
import xml
import xml.etree.ElementTree as ET

file = open('feedback.html', 'r')
content = file.read()
file.close()

# Parse the XML content
root = ET.fromstring(content)


In [4]:
# If you haven't installed BeautifulSoup, uncomment the next line:
# !pip install beautifulsoup4

from bs4 import BeautifulSoup

# Assume 'content' contains your HTML/XML string
soup = BeautifulSoup(content, 'html.parser')

feedback_entries = []
seen_entries = set()  # Track unique feedback entries to avoid duplicates

# Find all top-level divs that contain multiple feedback categories (one per reviewer)
# Looking for divs with class "jsx-8db43128a5365882" that contain multiple bordered divs
parent_divs = soup.find_all('div', class_='jsx-8db43128a5365882', recursive=True)

for parent in parent_divs:
    # Find all bordered divs within this parent (these are the individual feedback items)
    bordered_divs = parent.find_all('div', style=lambda v: v is not None and 'border' in v, recursive=False)
    
    # If we found multiple bordered divs, this is likely a feedback entry group
    if len(bordered_divs) >= 2:  # At least 2 categories means it's a real entry
        entry = {}
        entry_key_parts = []
        
        for block in bordered_divs:
            strong = block.find('strong')
            category = strong.get_text(strip=True).replace(':', '') if strong else 'Unknown'
            
            # Get the feedback text (span inside <p>)
            span = block.find('span')
            text = span.get_text(strip=True) if span else ''
            # Strip out newlines with multiple spaces and replace with single space
            text = text.replace('\n                ', ' ')
            
            if text:  # Only add non-empty feedback
                entry[category] = text
                entry_key_parts.append(text[:50])  # Use first 50 chars for uniqueness check
        
        # Only add if this entry has content and is unique
        entry_key = tuple(sorted(entry_key_parts))
        if entry and entry_key not in seen_entries:
            seen_entries.add(entry_key)
            feedback_entries.append(entry)

import json

# print to file as JSON
with open('feedback.json', 'w') as f:
    json.dump(feedback_entries, f, indent=2, ensure_ascii=False)

print(f"Extracted {len(feedback_entries)} unique feedback entries")

Extracted 33 unique feedback entries


In [ ]:
# !pip install openai

from openai import OpenAI
import json

# OpenRouter API setup
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
)

# Load feedback entries
with open('feedback.json', 'r') as f:
    feedback_entries = json.load(f)

summarized_feedback = []

print(f"Processing {len(feedback_entries)} feedback entries...")

for i, entry in enumerate(feedback_entries):
    print(f"\nProcessing entry {i+1}/{len(feedback_entries)}...")
    
    # Format the feedback for the prompt
    feedback_text = "\n".join([f"{cat}: {text}" for cat, text in entry.items()])
    
    # Create the prompt
    prompt = f"""Summarize this game feedback VERY briefly (1-2 words per category):

{feedback_text}

Respond ONLY with a JSON object with the same categories, each containing a 1-2 word summary."""
    
    try:
        # Make API request using OpenAI SDK
        completion = client.chat.completions.create(
            model="google/gemini-2.0-flash-exp:free",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        
        summary_text = completion.choices[0].message.content
        
        # Try to extract JSON from the response
        try:
            # Remove markdown code blocks if present
            if '```json' in summary_text:
                summary_text = summary_text.split('```json')[1].split('```')[0].strip()
            elif '```' in summary_text:
                summary_text = summary_text.split('```')[1].split('```')[0].strip()
            
            summary = json.loads(summary_text)
            summarized_feedback.append(summary)
            print(f"✓ Entry {i+1} summarized")
        except json.JSONDecodeError:
            print(f"✗ Entry {i+1} failed to parse: {summary_text[:100]}")
            summarized_feedback.append({"error": "Failed to parse summary"})
    except Exception as e:
        print(f"✗ Entry {i+1} API error: {str(e)}")
        summarized_feedback.append({"error": f"API error: {str(e)}"})

# Save summarized feedback
with open('feedback_summarized.json', 'w') as f:
    json.dump(summarized_feedback, f, indent=2, ensure_ascii=False)

print(f"\n✓ Complete! Saved {len(summarized_feedback)} summarized entries to feedback_summarized.json")

Processing 27 feedback entries...

Processing entry 1/27...
✗ Entry 1 API error: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-min. ', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '20', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1761182580000'}, 'provider_name': None}}, 'user_id': 'user_34RfvTvI0gbR89kZayT0SrjcUst'}

Processing entry 2/27...
✗ Entry 1 API error: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-min. ', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '20', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1761182580000'}, 'provider_name': None}}, 'user_id': 'user_34RfvTvI0gbR89kZayT0SrjcUst'}

Processing entry 2/27...
✗ Entry 2 API error: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-min. ', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '20', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1761182580000'}, 'provider_name': None

openrouter key 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